In [1]:
import json
import os
import shutil
import subprocess
import tempfile
import glob
import pandas as pd
from functools import reduce
import re
import pandas as pd
import time
from scipy import stats
import itertools

# reduce new function https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes

commonpeaksfile = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/allThePossiblePeaksnine.bed'

metadata = '/home/joaquin/projects/methylation/data/commonData/ids_data_allReplicates_methylation.json'
basePathDataFolder = '/home/joaquin/projects/methylation/data'
narrowPeakLocationFolders = ['tfs_rep_1','tfs_rep_3_input_from_rep_2', 'tfs_rep_2', 'tfs_rep_4']
specificPathsSumary = {}
with open(metadata) as jsonMetadata:
     experimentsClasification = json.load(jsonMetadata)['experiments']
for experiment in experimentsClasification:
    specificPathsSumary[experiment['condition']] = {}
    for metState in ['direct', 'amplified']:
        specificPathsSumary[experiment['condition']][metState] = {}
        for exptype in ['sample', 'input']:
            specificPathsSumary[experiment['condition']][metState][exptype] = {}
            for replicate, number in zip(
                experiment[metState],
                ['replicate1','replicate2', 'replicate3']
            ):
            
                expeId, expPath = replicate[number][0][exptype].strip().split(',')
        # if it is a missing experiment dont continue with the analisys
                if 'MISSING' in expeId:
                    print(number, expPath, expeId)
                    continue
                for possiblenarrowPeakFolder in narrowPeakLocationFolders:
                    narrowPeakFolder = None
                    path = os.path.join(basePathDataFolder,possiblenarrowPeakFolder,expPath)
        # make a list of the files in each posible directory. Try and continue if the experiment was not 
        # done for the replicate. 
                    try:
                        filesInFolder = os.listdir(path)
                    except FileNotFoundError:
                        continue
        # search for the specific experiment id inside the folder in the names to check if it is the correct folder
        # and stop searching if it is inside

                    for file in filesInFolder:
                        if 'html' in file:
                            fileid=file
    #                 print(fileid)
    #                 print(expeId)
                    if fileid.startswith(expeId):
                        narrowPeakFolder = possiblenarrowPeakFolder
                        break

                narrowpeakFileOriginalPath = os.path.join(
                    basePathDataFolder,narrowPeakFolder,expPath
                )
    #             specificPathsSumary[experiment['condition']][metState].append((narrowPeakFolder,narrowpeakFileOriginalPath))
                specificPathsSumary[experiment['condition']][metState][exptype][number] = narrowpeakFileOriginalPath 

replicate3 MYC3/amplified/6/Mock MISSING
replicate2 MYCH7/direct/24/Mock MISSING


In [2]:
def performShapiroTest(dfcol): #ej phenotypeValuesDf['notTransformed']
    # THE DATA IN ARRAY
#     phenotypeValuesDf = pd.DataFrame.from_dict(phenotypeValuesDict, orient='index', columns=['notTransformed'])
#     example for testing {'a': -2, 'b': 1, 'c': 3, 'd': 1, 'e': 2, 'f': 8}
    shapiro_test = stats.shapiro(dfcol)
    return shapiro_test.pvalue

def performPearsonCorr(datasetone,datasettwo):
    return stats.pearsonr(datasetone,datasettwo)

def generateDfandNormaliceDataTPMs(folder,dataColName):
    csvFile = glob.glob(f'{folder}/allThePossiblePeaksnine*.csv')
    
    if len(csvFile) != 1:
        return print(folder, ' has a problem selecting File')
    else:
        csvFilePath = csvFile[0]

    fileDf = pd.read_csv(csvFilePath,header=0, names=['chr','star','end','id',dataColName])
    totalReads = fileDf[dataColName].sum()

    scalingFactor = totalReads/100000

    fileDf[dataColName] = fileDf[dataColName].apply(lambda x: x/scalingFactor)

    return fileDf
    

In [3]:
# reduce new function https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes
import numpy as np
with open('datacorrgoodTPM100K.tsv', 'a+') as datacorr:
    for experiment in specificPathsSumary:
        for metState in specificPathsSumary[experiment]:
            for exptype in specificPathsSumary[experiment][metState]:
                listofdfs = []
                datacolnames = []
                for replicate in specificPathsSumary[experiment][metState][exptype]:                    
                    workingFolder = specificPathsSumary[experiment][metState][exptype][replicate]
                    datacolname = '{}{}{}{}'.format(experiment,metState,exptype,replicate)
                    replicateTPMdf = generateDfandNormaliceDataTPMs(workingFolder,datacolname)
#                     print(f'{datacolname} shapiro test is ', performShapiroTest(np.array(replicateTPMdf[datacolname])))
                    listofdfs.append(replicateTPMdf)
                    datacolnames.append(datacolname)
                    

    #             los valores que no estan en una de las replicas los completo con un 0
                df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['chr','star','end','id'],
                                                how='outer'), listofdfs).fillna(0)
                mean_col_name = '{}_{}_{}'.format(experiment,metState,exptype)
#                 df_merged[mean_col_name] = df_merged[datacolnames].mean(axis=1)
                listOfSubsets = []
                listOfSubsets.append(mean_col_name)
                for subset in itertools.combinations(datacolnames, 2):
                    corr, _ = performPearsonCorr(np.array(df_merged[subset[0]]), np.array(df_merged[subset[1]]))
                    listOfSubsets.append(f'{subset[0][-10:]}_vs_{subset[1][-10:]}:'+str(corr))
                datacorr.write('{}\n'.format('\t'.join(listOfSubsets)))
            #                 df_merged = df_merged.drop(columns=datacolnames)
                
#                 if not 'input' in mean_col_name:
#                     allNormalizedreplicates.append(df_merged)

# df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['chr','star','end','id'],
#                                                 how='outer'), allNormalizedreplicates).fillna(0)
  

In [ ]:

import math
import numpy as np
from scipy.stats import shapiro 
from scipy.stats import lognorm

#make this example reproducible
np.random.seed(1)

#generate dataset that contains 1000 log-normal distributed values
lognorm_dataset = lognorm.rvs(s=.5, scale=math.exp(1), size=8000)
shapiro(lognorm_dataset)

# ShapiroResult(statistic=0.8573324680328369, pvalue=3.880663073872444e-29)

In [ ]:
print(lognorm_dataset)

In [ ]:
for subset in itertools.combinations(specificPathsSumary[experimentCondition][metstate], 2):
            endProcess = subprocess.run(
                'bedtools intersect -wb -wa -a {} -b {} -f 0.6 -r'.format(subset[0], subset[1]),
                shell=True,
                capture_output=True
            )